In [1]:
from typing import List, Dict, Any, Optional, Callable
from abc import ABC, abstractmethod
import json
from dataclasses import dataclass
from dotenv import load_dotenv
load_dotenv()

@dataclass
class Message:
    role: str
    content: str
    metadata: Optional[Dict[str, Any]] = None

class LLMProvider(ABC):
    @abstractmethod
    def generate(self, prompt: str, **kwargs) -> str:
        pass

In [2]:
class OpenAIProvider(LLMProvider):
    def __init__(self, model: str = "gpt-4o-mini"):
        from openai import OpenAI
        self.client = OpenAI()
        self.model = model

    def generate(self, prompt: List[Message], **kwargs) -> tuple[str | dict, bool]:
        response = self.client.chat.completions.create(
            model=self.model,
            messages=prompt,
            **kwargs
        )
        if response.choices[0].message.tool_calls is None:
            return response.choices[0].message.content, False
        else:
            return response.choices[0].message, True
       

In [3]:
llm = OpenAIProvider()

In [28]:
import os
from tavily import TavilyClient
tavily_client = TavilyClient(api_key=f"{os.getenv('TAVILY_API_KEY')}")

def search_tavily(query, max_results=2, **kwargs):
    response = tavily_client.search(
        query=query,
        max_results=max_results,
        **kwargs        
    )
    return response['results']

def extract_tavily(urls, **kwargs):
    response = tavily_client.extract(
        urls=urls,
        **kwargs        
    )
    return response

In [5]:
tools = [{
    "type": "function",
    "function": {
        "name": "search_tavily",
        "description": "Search the web for a given query.",
        "parameters": {
            "type": "object",
            "properties": {
                "query": {
                    "type": "string",
                    "description": "query to search the web for"
                }
            },
            "required": [
                "query"
            ],
            "additionalProperties": False
        },
        "strict": True
    }
}]

In [7]:
class Tool:
    def __init__(
        self,
        description: List[Dict[str, Any]],
        function: Callable
    ):
        self.description = description
        self.function = function
   
        

In [54]:
class Agent:
    def __init__(
        self,
        llm: LLMProvider,
        system_prompt: str,
        tool: Tool = None
    ):
        self.llm = llm
        self.system_prompt = system_prompt
        self.messages = [{"role": "system", "content": self.system_prompt}]
        self.tool = tool

    def generate(self, prompt: str, **kwargs) -> str:
        self.messages.append({"role": "user", "content": prompt})

        if self.tool:
            completion, is_tool_call = self.llm.generate(
                prompt=self.messages,
                tools=self.tool.description
            )
            
            if is_tool_call:
                self.messages.append(completion)
                print('tool call start: ', self.messages)

                if len(completion.tool_calls) > 1:
                    for tool_call in completion.tool_calls:
                        args = json.loads(tool_call.function.arguments)
                        result = self.tool.function(args["query"])
                        print('search result: ', result)
                        self.messages.append({                             
                            "role": "tool",
                            "tool_call_id": tool_call.id,
                            "content": str(result)
                        })
                    print('multiple tool call: ', self.messages)
                else:
                    tool_call = completion.tool_calls[0]
                    args = json.loads(tool_call.function.arguments)
                    result = self.tool.function(args["query"])
                    print('search result: ', result)
                    self.messages.append({                             
                        "role": "tool",
                        "tool_call_id": tool_call.id,
                        "content": str(result)
                    })
                    print('single tool call: ', self.messages)
                    
                completion_2, _ = self.llm.generate(
                    prompt=self.messages,
                )

                return completion_2
            else:
                return completion
        else:
            completion, _ = self.llm.generate(
                prompt=self.messages                
            )
            return completion
        

In [48]:
search_tool = Tool(tools, search_tavily)

In [49]:
search_keyword_agent = Agent(
    llm=llm, 
    system_prompt="""
        사용자가 요청한 내용에 대한 검색 키워드 생성 어시스턴트입니다.
        너의 임무는 
         1. 요청한 내용에 대한 주장을 세분화 
         2. 각 주장에 대한 검색 키워드를 생성
        최적의 검색 결과를 얻기 위한 키워드를 생성해주세요.
        출력 형식은 다음과 같습니다.
        예시:
        지구는 평평하다 : 지구 평평
        미국은 중국을 넘어서 세계 최대 경제 강국이다 : 미국 중국 경제
    """    
)

In [55]:
search_agent = Agent(
    llm=llm, 
    system_prompt="""
        검색 키워드에 따라 도구를 사용해 인터넷 검색을 해주는 도우미입니다.
        출력 형식은 다음과 같습니다.
        예시: 
        지구는 평평하다 - 제목: 지구는 평평하다, 내용: 지구는 평평하다, 링크: https://www.google.com
        미국은 중국을 넘어서 세계 최대 경제 강국이다 - 제목: 미국은 중국을 넘어서 세계 최대 경제 강국이다, 내용: 미국은 중국을 넘어서 세계 최대 경제 강국이다, 링크: https://www.google.com
    """,
    tool=search_tool
)

In [51]:
verdict_agent = Agent(
    llm=llm, 
    system_prompt="""
        주어진 내용에 따라 사실인지 아닌 지 판단하고 그 근거를 내용에 기반해 제시해주는 도우미입니다. 링크도 포함해서 제시해주세요.
    """,
)

In [39]:
searched_keywords = search_keyword_agent.generate("지구는 평평하고 우주는 지구를 중심으로 돈다")
searched_keywords

'지구는 평평하다 : 지구 평평  \n우주는 지구를 중심으로 돈다 : 지구 중심 우주 회전  '

In [56]:
searched_result = search_agent.generate(searched_keywords)
searched_result

tool call start:  [{'role': 'system', 'content': '\n        검색 키워드에 따라 도구를 사용해 인터넷 검색을 해주는 도우미입니다.\n        출력 형식은 다음과 같습니다.\n        예시: \n        지구는 평평하다 - 제목: 지구는 평평하다, 내용: 지구는 평평하다, 링크: https://www.google.com\n        미국은 중국을 넘어서 세계 최대 경제 강국이다 - 제목: 미국은 중국을 넘어서 세계 최대 경제 강국이다, 내용: 미국은 중국을 넘어서 세계 최대 경제 강국이다, 링크: https://www.google.com\n    '}, {'role': 'user', 'content': '지구는 평평하다 : 지구 평평  \n우주는 지구를 중심으로 돈다 : 지구 중심 우주 회전  '}, ChatCompletionMessage(content=None, refusal=None, role='assistant', audio=None, function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_FqDSEBrOjpVIkr2zQ6L1NaKv', function=Function(arguments='{"query": "지구는 평평하다"}', name='search_tavily'), type='function'), ChatCompletionMessageToolCall(id='call_KDCh1tGCKRrCzL06BaowSP3m', function=Function(arguments='{"query": "우주는 지구를 중심으로 돈다"}', name='search_tavily'), type='function')])]
search result:  [{'title': '지평설 - 위키백과, 우리 모두의 백과사전', 'url': 'https://ko.wikipedia.org/wiki/지평설', 'content': "문서 문서 정보 기원전 4세기

'1. 지구는 평평하다\n   - 제목: 지구는 평평하다\n   - 내용: 지구는 평평하다는 주장은 역사적으로 그런 견해를 가진 사람들에게서 비롯되었습니다. 그러나 과학적 증거에 따르면 지구는 구형입니다.\n   - 링크: [지구 평평론 - 위키백과](https://ko.wikipedia.org/wiki/%EC%A7%80%ED%9B%88%EC%84%B1)\n\n2. 우주는 지구를 중심으로 돈다\n   - 제목: 우주는 지구를 중심으로 돈다\n   - 내용: 고대에는 지구가 우주의 중심이라고 믿는 지구 중심설이 있었습니다. 현재의 과학적 이해에 따르면 지구는 태양계의 행성 중 하나입니다.\n   - 링크: [지구와 사람들은 어떻게 돌았는가?](https://www.joongang.co.kr/article/3680596)'

In [53]:
verdict = verdict_agent.generate(searched_result)
verdict

'1. **지구는 평평하다** - **거짓**\n   - 근거: 지구의 형상에 대한 현대 과학의 이해에 따르면, 지구는 구형이며, 이는 아리스토텔레스(기원전 4세기 경)의 주장 등을 통해 역사적으로도 입증되었습니다. 평평하다는 주장은 과학적 증거에 의해 반박되어 왔습니다. \n   - 링크: [위키백과 - 지구](https://ko.wikipedia.org/wiki/%EC%A7%80%ED%9B%84%EC%84%A4)\n\n2. **우주는 지구를 중심으로 돈다** - **거짓**\n   - 근거: 과학적 연구와 관측을 통해 지구 이외의 많은 천체가 존재하며, 태양 주위에서 지구가 공전한다는 것은 알려진 사실입니다. 지구 중심적 사고는 역사적으로 존재했으나, 현대의 천문학에서는 이를 부정하고 있습니다. \n   - 링크: [중앙일보 기사](https://www.joongang.co.kr/article/3680596)'